# Produce the JSON syntax to specify the table columns in a SAP HANA Client in SAP Data Hub

Conventional connection to SAP HANA (outside SAP Data Hub)
https://blogs.sap.com/2017/07/26/sap-hana-2.0-sps02-new-feature-updated-python-driver/ 


In [1]:
# !pip install "C:\Program Files\SAP\hdbclient\hdbcli-2.3.119.zip"

In [2]:
from hdbcli import dbapi
dbapi.__name__

'hdbcli.dbapi'

In [3]:
# quick and dirty, should use hdbuserstore
conn = dbapi.connect(
    address="yourserver", 
    port=30015, 
    user="SYSTEM", 
    password="Welcome01"
)
# print(conn)

Specify the target table and retrieve its specifications

In [4]:
schemaName = "SYSTEM"
tableName = "VEHICLEREGISTRATIONS_FORECAST"
cursor = conn.cursor()
cursor.execute("SELECT COLUMN_NAME, DATA_TYPE_NAME, LENGTH FROM SYS.COLUMNS WHERE SCHEMA_NAME = '" + schemaName + "' AND TABLE_NAME = '" + tableName + "' ORDER BY POSITION;")
result_set = cursor.fetchall()

In [5]:
print(result_set)

[('SEGMENT', 'NVARCHAR', 14), ('MONTH', 'DATE', 10), ('MEASURETYPE', 'NVARCHAR', 20), ('MEASURE', 'DOUBLE', 15), ('MEASURELOWER', 'DOUBLE', 15), ('MEASUREUPPER', 'DOUBLE', 15), ('MODEL', 'NVARCHAR', 100)]


Turn the metadata into the appropriate JSON syntax

In [6]:
tableColumn = []
for row in result_set:
    if row[1] in ["VARCHAR", "NVARCHAR", "FLOAT"]:
        tableColumn.append({'name': row[0], 'type': row[1], 'size': row[2]})
    else:
        tableColumn.append({'name': row[0], 'type': row[1]})
cursor.close()
tableColumn

[{'name': 'SEGMENT', 'size': 14, 'type': 'NVARCHAR'},
 {'name': 'MONTH', 'type': 'DATE'},
 {'name': 'MEASURETYPE', 'size': 20, 'type': 'NVARCHAR'},
 {'name': 'MEASURE', 'type': 'DOUBLE'},
 {'name': 'MEASURELOWER', 'type': 'DOUBLE'},
 {'name': 'MEASUREUPPER', 'type': 'DOUBLE'},
 {'name': 'MODEL', 'size': 100, 'type': 'NVARCHAR'}]

Do not use the surrounding single '

In [7]:
import json
json_string = json.dumps(tableColumn)
json_string

'[{"name": "SEGMENT", "type": "NVARCHAR", "size": 14}, {"name": "MONTH", "type": "DATE"}, {"name": "MEASURETYPE", "type": "NVARCHAR", "size": 20}, {"name": "MEASURE", "type": "DOUBLE"}, {"name": "MEASURELOWER", "type": "DOUBLE"}, {"name": "MEASUREUPPER", "type": "DOUBLE"}, {"name": "MODEL", "type": "NVARCHAR", "size": 100}]'